# Importations and Required Libraries

In [20]:
pip install ipython-sql sqlalchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**load SQL magic and set up an SQLite database**

In [12]:
%load_ext sql
import sqlite3
import pandas as pd
from sqlalchemy import create_engine


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Step 1: Load Data into Pandas

In [13]:
# Load dataset files into DataFrames
df_active = pd.read_csv("Flipside Dataset/flipside_days_active.csv")
df_labels = pd.read_csv("Flipside Dataset/flipside_dim_labels.csv", delimiter=",", on_bad_lines="skip")
df_new_programs = pd.read_csv("Flipside_Dataset_2/Solana_Weekly_New_Programs.csv")
df_new_users = pd.read_csv("Flipside_Dataset_2/Solana Weekly New Users.csv")
df_unique_programs = pd.read_csv("Flipside_Dataset_2/Solana_Weekly_Unique_Programs.csv")
df_unique_users = pd.read_csv("Flipside_Dataset_2/Solana_Weekly_Unique_Users.csv")
df_days_since_last_use = pd.read_csv("Flipside Dataset/weekly_days_since_last_use.csv")
df_users_last_use = pd.read_csv("Flipside_Dataset_2/Solana_last_use_data.csv")

**create a database connection**

In [14]:
engine = create_engine('sqlite:///flipside_data.db')  
conn = engine.connect()

**Store DataFrames as SQL Tables**

In [16]:
df_active.to_sql('flipside_days_active', conn, if_exists='replace', index=False)
df_labels.to_sql('flipside_dim_labels', conn, if_exists='replace', index=False)
df_new_programs.to_sql('solana_weekly_new_programs', conn, if_exists='replace', index=False)
df_new_users.to_sql('solana_weekly_new_users', conn, if_exists='replace', index=False)
df_unique_programs.to_sql('solana_weekly_unique_programs', conn, if_exists='replace', index=False)
df_unique_users.to_sql('solana_weekly_unique_users', conn, if_exists='replace', index=False)
df_days_since_last_use.to_sql('weekly_days_since_last_use', conn, if_exists='replace', index=False)
df_users_last_use.to_sql('solana_last_use_data', conn, if_exists='replace', index=False)


100001

**Query using %%sql to inspect the tables**

In [23]:
%%sql 
sqlite:///flipside_data.db
SELECT * 
FROM solana_weekly_new_users 
LIMIT 5;

Done.


WEEK,NEW_USERS
2025-03-03 00:00:00.000,3.0
2025-02-03 00:00:00.000,1.0
2025-01-20 00:00:00.000,1.0
2025-01-13 00:00:00.000,2.0
2025-01-06 00:00:00.000,2.0


In [25]:
%%sql 
sqlite:///flipside_data.db
SELECT * 
FROM weekly_days_since_last_use 
LIMIT 5;

Done.


CREATION_DATE,Days since last use,Days since creation
2020-10-07,633.6165674428066,750.5566283125927
2020-10-14,611.4229036788537,743.5566283125927
2020-10-21,528.2639841669929,736.5566283125927
2020-10-28,492.9001518006344,729.5566283125927
2020-11-04,696.7685171327714,722.5566283125927


# Step 2: Merge Tables 


**I. Merge flipside_days_active, weekly_days_since_last_use, and solana_last_use_data on the ADDRESS column.**

In [ ]:
%%sql
SELECT 
    fda.ADDRESS, 
    fda.FIRST_TX_DATE, 
    fda.FIRST_PROGRAM_ID, 
    fda.LAST_TX_DATE, 
    fda.LAST_PROGRAM_ID, 
    fda.NUM_DAYS_ACTIVE, 
    fda.NUM_TXS, 
    fda.TOTAL_FEES, 
    fda.PROGRAMS_USED,
    wds.CREATION_DATE, 
    wds."Days since last use", 
    wds."Days since creation", 
    sld.LAST_USE
FROM flipside_days_active fda
LEFT JOIN weekly_days_since_last_use wds ON fda.ADDRESS = wds.CREATION_DATE
LEFT JOIN solana_last_use_data sld ON fda.ADDRESS = sld.ADDRESS;



 * sqlite:///flipside_data.db
Done.
